In [1]:
from yolo_detector import load_yolo_model, detect_image , draw_boxes
from util import write_results, prepare_image, get_detection_summary
from yolo_attacks import dag_attack, dag_disappearance_attack
import torch
from torchvision.utils import save_image

In [2]:
cfgfile = "cfg/yolov3.cfg"
reso = 416
weightsfile = "yolov3.weights"
images = "coco/test_images/000000000001.jpg"
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
CUDA = torch.cuda.is_available()
det = "detections"

In [3]:
model, classes, num_classes, inp_dim = load_yolo_model(cfgfile,weightsfile,reso ,CUDA = CUDA)

Loading network.....
Network successfully loaded


In [5]:
im_batches, im_dim_list, imlist, loaded_ims = prepare_image(images, inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, im_batches, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, det)
target_objects = get_detection_summary(output)
print(target_objects)

----------------------------------------------------------
Objects Detected:    car car truck parking meter
----------------------------------------------------------
Detection Images are saved in detections
[2, 12, 7]


In [6]:
adversarial_class = [0,4,5] #person and bicycle
adv_images_dag, _ = dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=20, 
    gamma=0.1, 
    CUDA=CUDA
)

Class ID: 2
Class ID: 2
Class ID: 7
Class ID: 12
Step 1/20: Loss = 471.17816162109375
Class ID: 2
Class ID: 7
Step 2/20: Loss = 4520.32861328125
Class ID: 0
Class ID: 2
Class ID: 7
Step 3/20: Loss = 7814.396484375
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 7
Class ID: 8
Step 4/20: Loss = -33.40653991699219
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 0
Class ID: 5
Class ID: 5
Iteration 5: Attack succeeded. Stopping early.
Saved adversarial image to: dag_adversarial_images/000000000001_adv_dag.png


In [7]:
adv_batch_dag, im_dim_list, imlist, loaded_ims = prepare_image("dag_adversarial_images", inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, adv_batch_dag, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "dag_adversarial_detections")
  

----------------------------------------------------------
Objects Detected:    person person person person person person person person bus bus
----------------------------------------------------------
Detection Images are saved in dag_adversarial_detections


In [8]:
#apply disappearance attack
adv_images_disappearance, _ = dag_disappearance_attack(
    model, 
    images, 
    target_objects, 
    max_iter=20, 
    gamma=0.1, 
    CUDA=CUDA
)

Class ID: 2
Class ID: 2
Class ID: 7
Class ID: 12
Step 1/20: Loss = -776.558837890625
Class ID: 2
Class ID: 7
Class ID: 12
Step 2/20: Loss = -605.2846069335938
Class ID: 2
Class ID: 7
Class ID: 12
Step 3/20: Loss = -576.605712890625
Class ID: 2
Class ID: 7
Class ID: 12
Step 4/20: Loss = -556.0577392578125
Class ID: 2
Class ID: 2
Class ID: 7
Class ID: 12
Step 5/20: Loss = -664.4259033203125
Class ID: 2
Class ID: 5
Class ID: 12
Step 6/20: Loss = -391.04327392578125
Class ID: 2
Class ID: 5
Class ID: 7
Class ID: 12
Step 7/20: Loss = -517.28125
Class ID: 2
Class ID: 5
Class ID: 7
Step 8/20: Loss = -314.5614013671875
Class ID: 2
Class ID: 5
Class ID: 7
Step 9/20: Loss = -319.78509521484375
Class ID: 2
Class ID: 5
Class ID: 7
Class ID: 12
Step 10/20: Loss = -496.9554138183594
Class ID: 0
Class ID: 0
Class ID: 2
Class ID: 2
Class ID: 5
Class ID: 7
Step 11/20: Loss = -459.8917541503906
Class ID: 0
Class ID: 2
Class ID: 2
Class ID: 5
Class ID: 7
Step 12/20: Loss = -433.4477844238281
Class ID: 0
C

In [9]:
adv_batch_disappearance, im_dim_list, imlist, loaded_ims = prepare_image("disappearance_adversarial_images", inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, adv_batch_disappearance, im_dim_list, imlist, loaded_ims, batch_size, 0.5, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "disappearance_adversarial_detections")

----------------------------------------------------------
Objects Detected:    person bus
----------------------------------------------------------
Detection Images are saved in disappearance_adversarial_detections
